In [1]:
import pandas
import psycopg2
import datetime
import matplotlib.pyplot as plt
import os


conn = psycopg2.connect(host = '127.0.0.1' , dbname = 'postgres' , user = 'postgres' , password = 'mon3636a')
cur = conn.cursor()
pandas.set_option('display.max_rows',1000)
plt.rcParams['font.sans-serif'] = ['Taipei Sans TC Beta']

## Argument

In [2]:
# 取最後交易日 , 讀取20日集中度報表
max_date = pandas.read_sql("select max(date) as date from work_date " , con = conn)['date'].iloc[0]

# 最後近20天交易日
date_list = tuple(pandas.read_sql("select * from work_date order by date desc limit 50" , con = conn)['date_no'])

## Create Folder

In [3]:
try :
    os.mkdir('/home/buneo/Stock/SLACK_REPORT/CCT20_SLACK/{}'.format(str(max_date)))
except FileExistsError:
    pass

try :
    os.mkdir('/home/buneo/Stock/SLACK_REPORT/CCT20_SLACK_MACD_GREEN/{}'.format(str(max_date)))
except FileExistsError:
    pass

try :
    os.mkdir('/home/buneo/Stock/SLACK_REPORT/CCT20_SLACK_C/{}'.format(str(max_date)))
except FileExistsError:
    pass

try :
    os.mkdir('/home/buneo/Stock/SLACK_REPORT/MACD_02/{}'.format(str(max_date)))
except FileExistsError:
    pass

try :
    os.mkdir('/home/buneo/Stock/SLACK_REPORT/MACD_24/{}'.format(str(max_date)))
except FileExistsError:
    pass

## CCT20 - Normal

In [4]:
# 20日集中度報表
cct20 = pandas.read_sql("select * from report.cct20_history where date ='{}'".format(str(max_date)) , con = conn)

# 20日集中度報表所有"股票代號"
no_list = cct20['no'].tolist()

In [5]:
for no in no_list:
    
    # 讀目前股票代號取近20日MACD
    no_macd = pandas.read_sql("select cast(date as varchar(10)) as date , no , macd_bar , macd , (mema-nema) as DIF from ods.macd where date_no in {0} and no = '{1}' order by date".format(date_list , no) , con = conn)
    no_macd = no_macd.reset_index()
    no_macd['color'] = no_macd.apply(lambda x : 'red' if x['macd_bar'] > 0 else 'green' , axis = 1)
    no_macd['xticks'] = no_macd.apply(lambda x : x['date'] if ((x['index']+1) % 5 == 0 or x['index'] == 0) else "" , axis = 1)   
    max_bar = no_macd['macd_bar'].abs().max()
    
    # 將20日集中度數值設定為變數
    tmp = cct20[cct20['no'] == no]
    cct20_no = tmp['no'].iloc[0]
    cct20_name = tmp['name'].iloc[0]
    cct20_date = tmp['date'].iloc[0]
    
    fig , ax = plt.subplots(1,1,figsize = (12,5))
    ax.set_title('{0} ({1}) '.format(cct20_no , cct20_name) , fontsize = 20)
    ax.set_ylim(-max_bar , max_bar)
    ax.grid(which = 'major')
    ax.bar(x = no_macd['date'] , height = no_macd['macd_bar'] , align = 'center' , color = no_macd['color'] )
    plt.xticks(no_macd['xticks'])
    plt.savefig('/home/buneo/Stock/SLACK_REPORT/CCT20_SLACK/{0}/{2}{1}.png'.format(max_date ,cct20_name , cct20_no))
    plt.close()

## CCT20 - Jheng

In [6]:
# 20日集中度報表
cct20 = pandas.read_sql("select * from report.cct20_history_c where date ='{}'".format(str(max_date)) , con = conn)

# 20日集中度報表所有"股票代號"
no_list = cct20['no'].tolist()

for no in no_list:
    
    # 讀目前股票代號取近20日MACD
    no_macd = pandas.read_sql("select cast(date as varchar(10)) as date , no , macd_bar , macd , (mema-nema) as DIF from ods.macd where date_no in {0} and no = '{1}' order by date".format(date_list , no) , con = conn)
    no_macd = no_macd.reset_index()
    no_macd['color'] = no_macd.apply(lambda x : 'red' if x['macd_bar'] > 0 else 'green' , axis = 1)
    no_macd['xticks'] = no_macd.apply(lambda x : x['date'] if ((x['index']+1) % 5 == 0 or x['index'] == 0) else "" , axis = 1)   
    max_bar = no_macd['macd_bar'].abs().max()
    
    # 將20日集中度數值設定為變數
    tmp = cct20[cct20['no'] == no]
    cct20_no = tmp['no'].iloc[0]
    cct20_name = tmp['name'].iloc[0]
    cct20_date = tmp['date'].iloc[0]
    
    fig , ax = plt.subplots(1,1,figsize = (12,5))
    ax.set_title('{0} ({1})'.format(cct20_no , cct20_name) , fontsize = 20)
    ax.set_ylim(-max_bar , max_bar)
    ax.grid(which = 'major')
    ax.bar(x = no_macd['date'] , height = no_macd['macd_bar'] , align = 'center' , color = no_macd['color'] )
    plt.xticks(no_macd['xticks'])
    plt.savefig('/home/buneo/Stock/SLACK_REPORT/CCT20_SLACK_C/{0}/{2}{1}.png'.format(max_date ,cct20_name, cct20_no))
    plt.close()

## MACD - Rising and OSC < 0

In [7]:
cct20 = pandas.read_sql("select * from ods.cct20 where date ='{}'".format(str(max_date)) , con = conn)

macd = pandas.read_sql("select no , date_no from ods.macd where date = '{0}' and macd_bar < 0".format(max_date) , con = conn)

green_macd_no = macd['no'].tolist()
date_no = macd['date_no'].unique()[0]-3

main = pandas.read_sql("select * from ods.macd where date_no >= '{0}' and no in {1} order by no , date".format(date_no,tuple(green_macd_no)), con = conn)
main['bigger'] = ''

for i in range(1,len(main)):#range(len(main)):
    if main['no'][i] == main['no'][i-1] :
        if main['macd_bar'][i] >= main['macd_bar'][i-1]:
            main['bigger'][i] = main['bigger'][i-1]+'Y'
    else :
        main['bigger'][i] = ''

rise_no = main[main['bigger'] == 'YYY'][['no']]
no_5d = pandas.read_sql("select no , count(*) from (\
                             select no from stock_daily s join work_date d on s.date = d.date and d.date_no >= {0}\
                             union all \
                             select no from   otc_daily o join work_date d on o.date = d.date and d.date_no >= {0}) a\
                             group by no having count(*) = 5".format(date_no) , con = conn)['no'].tolist()
vol_1000 = pandas.read_sql("select no , avg(qty) from (select no , quantity qty from stock_daily s join work_date d on s.date = d.date and d.date_no >= {0} \
                                                       union all \
                                                       select no , quantity qty from   otc_daily o join work_date d on o.date = d.date and d.date_no >= {0} ) a \
                            group by no having avg(qty) >= 1000".format(date_no) , con = conn)
image_no = pandas.merge(left = rise_no , right = vol_1000 , on = 'no')

price_no = pandas.read_sql("select date , no from ods.analyze_avg where date_no = '{0}' and close > sma60".format(date_no+3) , con = conn)[['no']]

image_no = pandas.merge(left = image_no , right = price_no , on = 'no')['no'].tolist()


for no in image_no:

    # 讀目前股票代號取近20日MACD
    no_macd = pandas.read_sql("select cast(date as varchar(10)) as date , no , macd_bar , macd , (mema-nema) as DIF from ods.macd where date_no in {0} and no = '{1}' order by date".format(date_list , no) , con = conn)
    no_macd = no_macd.reset_index()
    no_macd['color'] = no_macd.apply(lambda x : 'red' if x['macd_bar'] > 0 else 'green' , axis = 1)
    no_macd['xticks'] = no_macd.apply(lambda x : x['date'] if ((x['index']+1) % 5 == 0 or x['index'] == 0) else "" , axis = 1)   
    max_bar = no_macd['macd_bar'].abs().max()
    
    # 將20日集中度數值設定為變數
    tmp = cct20[cct20['no'] == no]
    cct20_no = tmp['no'].iloc[0]
    cct20_name = pandas.read_sql("select no , name from stock_daily where date = '{0}' and no = '{1}' union select no , name from otc_daily where date = '{0}' and no = '{1}'".format(max_date , no) , con = conn)['name'].iloc[0]
    cct20_date = tmp['date'].iloc[0]
    
    fig , ax = plt.subplots(1,1,figsize = (12,5))
    ax.set_title('{0} ({1})'.format(cct20_no , cct20_name) , fontsize = 20)
    ax.set_ylim(-max_bar , max_bar)
    ax.grid(which = 'major')
    ax.bar(x = no_macd['date'] , height = no_macd['macd_bar'] , align = 'center' , color = no_macd['color'] )
    plt.xticks(no_macd['xticks'])
    plt.savefig('/home/buneo/Stock/SLACK_REPORT/CCT20_SLACK_MACD_GREEN/{0}/{2}{1}.png'.format(max_date ,cct20_name , cct20_no))
    plt.close()

<ipython-input-7-b8bbd27c9724>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main['bigger'][i] = main['bigger'][i-1]+'Y'
<ipython-input-7-b8bbd27c9724>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main['bigger'][i] = ''


## MACD - 0 >= OSC >= -2

In [8]:
vol_600 = pandas.read_sql("select no , avg(qty) from (select no , quantity qty from stock_daily s join work_date d on s.date = d.date and d.date_no >= {0} \
                                                       union all \
                                                       select no , quantity qty from   otc_daily o join work_date d on o.date = d.date and d.date_no >= {0} ) a \
                            group by no having avg(qty) >= 600".format(date_no) , con = conn)
price_no = pandas.read_sql("select date , no from ods.analyze_avg where date = '{0}' and close > 15 and close < 300 ".format(max_date) , con = conn)[['no']]

In [9]:
macd_02 = pandas.read_sql("select no , date_no from ods.macd where date = '{0}' and macd_bar <= 0 and macd_bar >= -2 ".format(max_date) , con = conn)
macd_02 = pandas.merge(macd_02 , vol_600 , on = 'no')
macd_02 = pandas.merge(macd_02 , price_no , on = 'no')['no'].tolist()

for no in macd_02:

    # 讀目前股票代號取近20日MACD
    no_macd = pandas.read_sql("select cast(date as varchar(10)) as date , no , macd_bar , macd , (mema-nema) as DIF from ods.macd where date_no in {0} and no = '{1}' order by date".format(date_list , no) , con = conn)
    no_macd = no_macd.reset_index()
    no_macd['color'] = no_macd.apply(lambda x : 'red' if x['macd_bar'] > 0 else 'green' , axis = 1)
    no_macd['xticks'] = no_macd.apply(lambda x : x['date'] if ((x['index']+1) % 5 == 0 or x['index'] == 0) else "" , axis = 1)   
    max_bar = no_macd['macd_bar'].abs().max()
    
    # 將20日集中度數值設定為變數
    tmp = cct20[cct20['no'] == no]
    cct20_no = tmp['no'].iloc[0]
    cct20_name = pandas.read_sql("select no , name from stock_daily where date = '{0}' and no = '{1}' union select no , name from otc_daily where date = '{0}' and no = '{1}'".format(max_date , no) , con = conn)['name'].iloc[0]
    cct20_date = tmp['date'].iloc[0]
    
    fig , ax = plt.subplots(1,1,figsize = (12,5))
    ax.set_title('{0} ({1})'.format(cct20_no , cct20_name) , fontsize = 20)
    ax.set_ylim(-max_bar , max_bar)
    ax.grid(which = 'major')
    ax.bar(x = no_macd['date'] , height = no_macd['macd_bar'] , align = 'center' , color = no_macd['color'] )
    plt.xticks(no_macd['xticks'])
    plt.savefig('/home/buneo/Stock/SLACK_REPORT/MACD_02/{0}/{2}{1}.png'.format(max_date ,cct20_name , cct20_no))
    plt.close()
    

## MACD - 2 >= OSC >= -4

In [10]:
macd_24 = pandas.read_sql("select no , date_no from ods.macd where date = '{0}' and macd_bar <= -2 and macd_bar >= -4 ".format(max_date) , con = conn)
macd_24 = pandas.merge(macd_24 , vol_600 , on = 'no')
macd_24 = pandas.merge(macd_24 , price_no , on = 'no')['no'].tolist()

for no in macd_24:

    # 讀目前股票代號取近20日MACD
    no_macd = pandas.read_sql("select cast(date as varchar(10)) as date , no , macd_bar , macd , (mema-nema) as DIF from ods.macd where date_no in {0} and no = '{1}' order by date".format(date_list , no) , con = conn)
    no_macd = no_macd.reset_index()
    no_macd['color'] = no_macd.apply(lambda x : 'red' if x['macd_bar'] > 0 else 'green' , axis = 1)
    no_macd['xticks'] = no_macd.apply(lambda x : x['date'] if ((x['index']+1) % 5 == 0 or x['index'] == 0) else "" , axis = 1)   
    max_bar = no_macd['macd_bar'].abs().max()
    
    # 將20日集中度數值設定為變數
    tmp = cct20[cct20['no'] == no]
    cct20_no = tmp['no'].iloc[0]
    cct20_name = pandas.read_sql("select no , name from stock_daily where date = '{0}' and no = '{1}' union select no , name from otc_daily where date = '{0}' and no = '{1}'".format(max_date , no) , con = conn)['name'].iloc[0]
    cct20_date = tmp['date'].iloc[0]
    
    fig , ax = plt.subplots(1,1,figsize = (12,5))
    ax.set_title('{0} ({1})'.format(cct20_no , cct20_name) , fontsize = 20)
    ax.set_ylim(-max_bar , max_bar)
    ax.grid(which = 'major')
    ax.bar(x = no_macd['date'] , height = no_macd['macd_bar'] , align = 'center' , color = no_macd['color'] )
    plt.xticks(no_macd['xticks'])
    plt.savefig('/home/buneo/Stock/SLACK_REPORT/MACD_24/{0}/{2}{1}.png'.format(max_date ,cct20_name , cct20_no))
    plt.close()